In [1]:
import pandas as pd
from tqdm import tqdm 
import os

In [2]:
annotations_df = pd.read_csv("F:\\DATA\\JSmith_SAFER_20220310\\raw_data\\Feas1\\rec_data_anon.csv") 
diag_key_df = pd.read_csv("F:\\DATA\\JSmith_SAFER_20220310\\raw_data\\Feas1\\rec_data_anon_diag_key.csv") 

In [3]:
annotations_df

,ptID,age,gender,noWeeks,ptDiag,ptDiagRev1,ptDiagRev2,ptDiagRev3,cardRev,measNo,...,tag_orig_Poor_Quality,tag_orig_Slow_Episode,tag_orig_Slow_Regular,tag_orig_Trigemini,tag_orig_Wide_QRS,poss_AF_tag,not_tagged,presNoAF,presAF,measID
0,1,69.0,1,2,4,6,6,6,0,1,...,0,0,0,0,0,0,1,1,0,1
1,1,69.0,1,2,4,6,6,6,0,2,...,0,0,0,0,0,0,1,1,0,2
2,1,69.0,1,2,4,6,6,6,0,3,...,0,0,0,0,0,0,1,1,0,3
3,1,69.0,1,2,4,6,6,6,0,4,...,0,0,0,0,0,1,0,0,0,4
4,1,69.0,1,2,4,6,6,6,0,5,...,0,0,0,0,0,0,1,1,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162426,2132,65.0,1,2,4,6,6,6,0,50,...,0,0,0,0,0,0,1,1,0,162427
162427,2132,65.0,1,2,4,6,6,6,0,51,...,1,0,0,0,0,0,0,0,0,162428
162428,2132,65.0,1,2,4,6,6,6,0,52,...,0,0,0,0,0,0,1,1,0,162429
162429,2132,65.0,1,2,4,6,6,6,0,53,...,0,0,0,0,0,0,1,1,0,162430


In [4]:
diag_key_df

,txt,num
0,2nd or 3rd degree heart block,1
1,AF >= 30 seconds,2
2,"Cannot exclude AF>=30 seconds, VT or 2/3 degre...",3
3,"No AF >= 30 secs, VT or 2/3 degree heart block...",4
4,Screening Failure,5
5,Undecided,6
6,Ventricular tachycardia,7
7,Disagreement between reviewers,-1


In [5]:
decode_diagnosis_dict = {k: v for k, v in zip(diag_key_df.num.to_list(),diag_key_df.txt.to_list())} 

In [6]:
all_files = []
for root, dirs, files in tqdm(os.walk("F:\\DATA\\JSmith_SAFER_20220310\\raw_data\\Feas1\\ECGs")):
    filetype = ".dat"
    for file in files:
        if file.endswith(filetype):
            all_files.append(os.path.join(root,file.split(filetype,2)[0]))

164it [00:01, 99.31it/s] 


In [7]:
af_files = []
maybe_af_files = []
normal_files = []

for file in tqdm(all_files):
    measID = int(file.split("_")[-1])
    annotations_df_row_for_file = annotations_df[annotations_df.measID == measID]
    diag_ = annotations_df_row_for_file.measDiag.to_list()[0]
    diag_rev1 = annotations_df_row_for_file.measDiagRev1.to_list()[0]
    diag_rev2 = annotations_df_row_for_file.measDiagRev2.to_list()[0]
    all_diag = [diag_, diag_rev1, diag_rev2]
    diagnosis_code_for_file = (2 if any([diag==2 for diag in all_diag]) else diag_) 
    if diagnosis_code_for_file == 2: af_files.append([file])
    if diagnosis_code_for_file == 3: maybe_af_files.append([file])
    if diagnosis_code_for_file == 4: normal_files.append([file])
    # if not [6,6,6] == all_diag:
    #     print(all_diag)
    # if diagnosis_code_for_file != 6:
    #     print(diagnosis_code_for_file)


100%|████████████████████████████████████████████████████████████████████████| 162365/162365 [01:32<00:00, 1757.71it/s]

Stored 'af_files' (list)
Stored 'maybe_af_files' (list)
Stored 'normal_files' (list)


In [9]:
af_files[0]

['F:\\DATA\\JSmith_SAFER_20220310\\raw_data\\Feas1\\ECGs\\002000\\saferF1_002958']

In [10]:
len(af_files), len(maybe_af_files), len(normal_files)

(906, 486, 3118)

In [11]:
import csv

with open("F:\\DATA\\JSmith_SAFER_20220310\\raw_data\\Feas1\\files_with_af.csv", 'w') as myfile:
    write = csv.writer(myfile) 
    write.writerows(af_files) 
    
with open("F:\\DATA\\JSmith_SAFER_20220310\\raw_data\\Feas1\\files_with_maybe_af.csv", 'w', newline='') as myfile:
    write = csv.writer(myfile) 
    write.writerows(maybe_af_files) 

with open("F:\\DATA\\JSmith_SAFER_20220310\\raw_data\\Feas1\\files_with_normal.csv", 'w', newline='') as myfile:
    write = csv.writer(myfile) 
    write.writerows(normal_files) 

## Write script to load all the data from the lists created, for running NNN on 

In [12]:
pd.read_csv("F:\\DATA\\JSmith_SAFER_20220310\\raw_data\\Feas1\\files_with_af.csv", names=["filepath"]).filepath.to_list()


['F:\\DATA\\JSmith_SAFER_20220310\\raw_data\\Feas1\\ECGs\\002000\\saferF1_002958',
 'F:\\DATA\\JSmith_SAFER_20220310\\raw_data\\Feas1\\ECGs\\003000\\saferF1_003519',
 'F:\\DATA\\JSmith_SAFER_20220310\\raw_data\\Feas1\\ECGs\\003000\\saferF1_003542',
 'F:\\DATA\\JSmith_SAFER_20220310\\raw_data\\Feas1\\ECGs\\003000\\saferF1_003566',
 'F:\\DATA\\JSmith_SAFER_20220310\\raw_data\\Feas1\\ECGs\\003000\\saferF1_003588',
 'F:\\DATA\\JSmith_SAFER_20220310\\raw_data\\Feas1\\ECGs\\003000\\saferF1_003589',
 'F:\\DATA\\JSmith_SAFER_20220310\\raw_data\\Feas1\\ECGs\\003000\\saferF1_003597',
 'F:\\DATA\\JSmith_SAFER_20220310\\raw_data\\Feas1\\ECGs\\003000\\saferF1_003621',
 'F:\\DATA\\JSmith_SAFER_20220310\\raw_data\\Feas1\\ECGs\\003000\\saferF1_003622',
 'F:\\DATA\\JSmith_SAFER_20220310\\raw_data\\Feas1\\ECGs\\003000\\saferF1_003623',
 'F:\\DATA\\JSmith_SAFER_20220310\\raw_data\\Feas1\\ECGs\\003000\\saferF1_003764',
 'F:\\DATA\\JSmith_SAFER_20220310\\raw_data\\Feas1\\ECGs\\003000\\saferF1_003765',
 'F: